In [ ]:
# AutoGluon Text 분석
# https://auto.gluon.ai/stable/tutorials/multimodal/text_prediction/beginner_text.html#sentiment-analysis-task

# Hyperparmeter 최적화
# https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html

In [ ]:
!pip3 install -U pip
!pip3 install -U setuptools wheel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.8 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.5.0
    Uninstalling setuptools-67.5.0:
      Successfully uninstalled setuptools-67.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
cvxpy 1.2.3 requires setuptools<=64.0.2, but you have setuptools 67.5.1 which is incompatible.


In [ ]:
!pip3 install torch==1.13.1+cpu torchvision==0.14.1+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html
!pip3 install torch==1.13.1+cu116 torchvision==0.14.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cpu/torch_stable.html
  Using cached https://download.pytorch.org/whl/cpu/torch-1.13.1%2Bcpu-cp38-cp38-linux_x86_64.whl (199.1 MB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.14.1%2Bcpu-cp38-cp38-linux_x86_64.whl (16.8 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.1+cu116
    Uninstalling torchvision-0.14.1+cu116:
      Successfully uninstalled torchvision-0.14.1+cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
  Using cached https://download.pytorch.org/whl/cu116/torch-1.13.1%2Bcu116-cp38-cp38-linux_x86_64.whl (1

In [ ]:
!pip3 install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

PATH = "/content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/"

train = pd.read_csv(PATH+'data/train.csv')
test = pd.read_csv(PATH+'data/test.csv')
submission = pd.read_csv(PATH+'data/sample_submission.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train

,ID,prompt,answer,AI
0,690.0,What is the future of AI?,The future of Artificial Intelligence (AI) is ...,1.0
1,304.0,What is your biggest challenge in your career?,My biggest challenge in my career is balancing...,0.0
2,63.0,What is the tallest mountain in the world?,The tallest mountain in the world is Mount Eve...,1.0
3,894.0,What are the best 5 super cars?,McLaren 720S.\r\nFerrari 296 GTB.\r\nAudi R8.\...,0.0
4,796.0,What is deep learning?,a type of machine learning based on artificial...,0.0
...,...,...,...,...
743,283.0,What is the most important thing in a family?,The most important thing in a family for me is...,0.0
744,895.0,What are the best machine learning Models?,Linear regression.\r\nLogistic regression\r\nD...,0.0
745,963.0,How can I chop onions without crying?,The best thing to do is very simple: use a ver...,0.0
746,156.0,What is the best way to stay healthy?,The best way to stay healthy varies from perso...,0.0


## 1. 전처리

## 1-1. null값 처리

In [ ]:
train["prompt"].isna().sum()

0

In [ ]:
# train = train[train["prompt"].isna()==False]

In [ ]:
train["answer"].isna().sum()

1

In [ ]:
train = train[train["answer"].isna()==False]

In [ ]:
train["AI"].isna().sum()

1

In [ ]:
train = train[train["AI"].isna()==False]

In [ ]:
test.isna().sum()

ID        0
prompt    0
answer    0
dtype: int64

In [ ]:
train

,ID,prompt,answer,AI
0,690.0,What is the future of AI?,The future of Artificial Intelligence (AI) is ...,1.0
1,304.0,What is your biggest challenge in your career?,My biggest challenge in my career is balancing...,0.0
2,63.0,What is the tallest mountain in the world?,The tallest mountain in the world is Mount Eve...,1.0
3,894.0,What are the best 5 super cars?,McLaren 720S.\r\nFerrari 296 GTB.\r\nAudi R8.\...,0.0
4,796.0,What is deep learning?,a type of machine learning based on artificial...,0.0
...,...,...,...,...
743,283.0,What is the most important thing in a family?,The most important thing in a family for me is...,0.0
744,895.0,What are the best machine learning Models?,Linear regression.\r\nLogistic regression\r\nD...,0.0
745,963.0,How can I chop onions without crying?,The best thing to do is very simple: use a ver...,0.0
746,156.0,What is the best way to stay healthy?,The best way to stay healthy varies from perso...,0.0


## 1-2. 데이터셋 추가

In [ ]:
import json
import os
curl = """
curl -X GET "https://datasets-server.huggingface.co/first-rows?dataset=Hello-SimpleAI%2FHC3&config=wiki_csai&split=train"
"""

In [ ]:
# run curl comand and load response --> json dict --> df
wiki_csai = json.loads(os.popen(curl).read())

In [ ]:
cols = [x['name'] for x in wiki_csai['features']]
rows = [x['row'].values() for x in wiki_csai['rows']]
wiki_data = pd.DataFrame(rows, columns=cols)

In [ ]:
wiki_df = pd.DataFrame([])
wiki_df['ID'] = [-1 for _ in range(len(wiki_data)*2)]
wiki_df['prompt'] = list(wiki_data['question'])*2
wiki_df['answer'] = [''.join(x) for x in wiki_data['human_answers']] + [''.join(x) for x in wiki_data['chatgpt_answers']]
wiki_df['AI'] = [0 for _ in range(len(wiki_data))] + [1 for _ in range(len(wiki_data))]
wiki_df

,ID,prompt,answer,AI
0,-1,"Please explain what is ""Animal cognition""",Animal cognition encompasses the mental capaci...,0
1,-1,"Please explain what is ""Human intelligence""",Human intelligence is the intellectual capabil...,0
2,-1,"Please explain what is ""Oxford English Diction...",The Oxford English Dictionary (OED) is the fir...,0
3,-1,"Please explain what is ""Oxford University Press""",Oxford University Press (OUP) is the universit...,0
4,-1,"Please explain what is ""AI applications""",Artificial intelligence (AI) has been used in ...,0
...,...,...,...,...
149,-1,"Please explain what is ""Optimization theory""",Optimization theory is a branch of mathematics...,1
150,-1,"Please explain what is ""Natural-language user ...",A natural-language user interface (NLUI) is a ...,1
151,-1,"Please explain what is ""Information retrieval""",Information retrieval (IR) is the process of s...,1
152,-1,"Please explain what is ""Question answering""",Question answering is a type of natural langua...,1


In [ ]:
all_train = pd.concat([train, wiki_df], axis=0)

In [ ]:
all_train

,ID,prompt,answer,AI
0,690.0,What is the future of AI?,The future of Artificial Intelligence (AI) is ...,1.0
1,304.0,What is your biggest challenge in your career?,My biggest challenge in my career is balancing...,0.0
2,63.0,What is the tallest mountain in the world?,The tallest mountain in the world is Mount Eve...,1.0
3,894.0,What are the best 5 super cars?,McLaren 720S.\r\nFerrari 296 GTB.\r\nAudi R8.\...,0.0
4,796.0,What is deep learning?,a type of machine learning based on artificial...,0.0
...,...,...,...,...
149,-1.0,"Please explain what is ""Optimization theory""",Optimization theory is a branch of mathematics...,1.0
150,-1.0,"Please explain what is ""Natural-language user ...",A natural-language user interface (NLUI) is a ...,1.0
151,-1.0,"Please explain what is ""Information retrieval""",Information retrieval (IR) is the process of s...,1.0
152,-1.0,"Please explain what is ""Question answering""",Question answering is a type of natural langua...,1.0


## 1-3. 데이터 정제

In [ ]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np

In [ ]:
import re
import emoji
from unidecode import unidecode

#remove html tags
def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)
    
#remove emoji and symbols
def dataPreprocessing(x): 
    x = x.lower()
    x = removeHTML(x)
    x = emoji.demojize(x, delimiters=(" ", " "))
    x = re.sub("'re", ' are',x) # 're n't --> are not 
    x = re.sub("n't", ' not',x) # 're n't --> are not 
    x = re.sub(r'\([^)]*\)', '' ,x)
    x = re.sub("/n",' ',x) # index 3번 \r, \n 삭제
    x = re.sub("/r",' ',x) # index 3번 \r, \n 삭제
    x = re.sub(r'[^A-Za-z0-9!.?, ]', '', x) 
    # x = re.sub(r'[!.?]','/n',x) # removed to try sentence structure analysis
    x = re.sub(r'[,]',' ',x)
    # x = re.sub("@\w+", '',x) # removing mentions (@)
    # x = re.sub("'\d+", '',x)
    # x = re.sub("\d+", '',x)
    # x = re.sub(r"[^\w\s]", '',x) # to remove symbols
    # x = re.sub("http\w+", '',x)
    # x = re.sub("\s[a-z]\s", '',x)
    x = re.sub("[ ]+",' ',x) # space*n --> space
    x = re.sub("_",'',x) # _people_ --> people
    x = x.strip()
    return unidecode(x) #emily bronté --> emily bronte, mi² --> mi2, πr --> pr

In [ ]:
train_text = all_train.copy()
train_text["prompt"]=train["prompt"].apply(lambda x: dataPreprocessing(x))
train_text["answer"]=train["answer"].apply(lambda x: dataPreprocessing(x))

In [ ]:
test_text = test.copy()
test_text["prompt"]=test["prompt"].apply(lambda x: dataPreprocessing(x))
test_text["answer"]=test["answer"].apply(lambda x: dataPreprocessing(x))

In [ ]:
train_text

,ID,prompt,answer,AI
0,690.0,what is the future of ai?,the future of artificial intelligence is both ...,1.0
1,304.0,what is your biggest challenge in your career?,my biggest challenge in my career is balancing...,0.0
2,63.0,what is the tallest mountain in the world?,the tallest mountain in the world is mount eve...,1.0
3,894.0,what are the best 5 super cars?,mclaren 720s.ferrari 296 gtb.audi r8.maserati ...,0.0
4,796.0,what is deep learning?,a type of machine learning based on artificial...,0.0
...,...,...,...,...
149,-1.0,what is the name of the largest rainforest in ...,the amazon rainforest,1.0
150,-1.0,what is the best way to handle change?,the best way to handle change is to stay posit...,1.0
151,-1.0,what is the history of ai?,the beginnings of modern ai can be traced to c...,1.0
152,-1.0,what is the difference between a static method...,a static method is a method that belongs to a ...,1.0


In [ ]:

# from sklearn.model_selection import train_test_split

# X = train_text.drop(['ID'], axis=1)
# y = train['AI']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# train_data = pd.concat([pd.DataFrame(X_train, columns=['prompt', 'answer']), pd.DataFrame(y_train, columns=['AI'])], axis=1)
train_data = train_text.copy().drop(['ID'], axis=1)
train_data

,prompt,answer,AI
0,what is the future of ai?,the future of artificial intelligence is both ...,1.0
1,what is your biggest challenge in your career?,my biggest challenge in my career is balancing...,0.0
2,what is the tallest mountain in the world?,the tallest mountain in the world is mount eve...,1.0
3,what are the best 5 super cars?,mclaren 720s.ferrari 296 gtb.audi r8.maserati ...,0.0
4,what is deep learning?,a type of machine learning based on artificial...,0.0
...,...,...,...
149,what is the name of the largest rainforest in ...,the amazon rainforest,1.0
150,what is the best way to handle change?,the best way to handle change is to stay posit...,1.0
151,what is the history of ai?,the beginnings of modern ai can be traced to c...,1.0
152,what is the difference between a static method...,a static method is a method that belongs to a ...,1.0


In [ ]:
test_data = test_text.copy().drop(['ID'], axis=1)
test_data

,prompt,answer
0,whats the best fast food,as i mentioned earlier best fast food is subje...
1,what is jsx in react?,jsx is a syntax extension for javascript that ...
2,what is the name of the reaction where a subst...,the reaction where a substance combines with o...
3,what is your opinion on the current state of t...,the current state of the environment is a caus...
4,what is deep learning?,deep learning is a subfield of machine learnin...
...,...,...
244,what is the name of the largest mammal that ca...,is the pterosaur a species of flying reptiles ...
245,what are the features of angular?,some of the features of angular include twoway...
246,what is the name of the largest salt flat in t...,salar de uyuni
247,who wrote the book to kill a mockingbird?,harper lee.


## 2. 모델링

In [ ]:
label = 'AI'
eval_metric = 'acc'
time_limit = 3600 * 2

In [ ]:
from ray import tune

In [ ]:
hyperparameters = {
        # "optimization.learning_rate": tune.uniform(0.00005, 0.005),
        # "optimization.optim_type": tune.choice(["adamw", "sgd"]),
        # "optimization.max_epochs": tune.choice(["40", "50"]),
        # "model.timm_image.checkpoint_name": tune.choice(["swin_base_patch4_window7_224", "convnext_base_in22ft1k"])
}
hyperparameter_tune_kwargs = {
    # "searcher": "bayes", # random
    # "scheduler": "ASHA",
    # "num_trials": 2,
}

In [ ]:
from autogluon.multimodal import MultiModalPredictor
# 모듈 못찾는 Attribute Error 발생시 런타임 초기화 후 코드 처음부터 재실행
# EX. AttributeError: module 'PIL.Image' has no attribute 'Resampling'
import uuid

# model_path = PATH+f"model/{uuid.uuid4().hex}-automm_sst_QA_unicode_2hours"
# predictor = MultiModalPredictor(label=label, eval_metric=eval_metric, path=model_path)
predictor = MultiModalPredictor(label=label, eval_metric=eval_metric)
predictor.fit(train_data,presets='best_quality', 
              hyperparameters={"optimization.max_epochs": 1000, "optimization.patience": 500},
              time_limit=time_limit)

INFO:lightning_fabric.utilities.seed:Global seed set to 123
No path specified. Models will be saved in: "AutogluonModels/ag-20230306_125146/"
AutoMM starts to create your model. ✨

- Model will be saved to "/content/AutogluonModels/ag-20230306_125146".

- Validation metric is "acc".

- To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /content/AutogluonModels/ag-20230306_125146
    ```

Enjoy your coffee, and let AutoMM do the job ☕☕☕ Learn more at https://auto.gluon.ai



/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit None Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 49, global step 300: 'val_acc' was not in top 3


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 50, global step 302: 'val_acc' was not in top 3
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
Start to fuse 3 checkpoints via the greedy soup algorithm.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/autogluon/multimodal/predictor.py:825 in fit              │
│                                                                                                  │
│    822 │   │   │   )                                                                             │
│    823 │   │   │   return predictor                                                              │
│    824 │   │                                                                                     │
│ ❱  825 │   │   self._fit(**_fit_args)                                                            │
│    826 │   │   training_end = time.time()                                                        │
│    827 │   │   self._total_train_time = training_end - training_start                            │
│    828                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/autogluon/multimodal/predictor.py:1395 in _fit            │
│                                                                                                  │
│   1392 │   │   │   # We do not perform averaging checkpoint in the case of hpo for each trial    │
│   1393 │   │   │   # We only averaging the checkpoint of the best trial in the end in the maste  │
│   1394 │   │   │   if not hpo_mode:                                                              │
│ ❱ 1395 │   │   │   │   self._top_k_average(                                                      │
│   1396 │   │   │   │   │   model=model,                                                          │
│   1397 │   │   │   │   │   save_path=save_path,                                                  │
│   1398 │   │   │   │   │   minmax_mode=minmax_mode,                                              │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/autogluon/multimodal/predictor.py:1474 in _top_k_average  │
│                                                                                                  │
│   1471 │   │   │   │   │   │   │   f"Start to fuse {len(top_k_model_paths)} checkpoints via the  │
│   1472 │   │   │   │   │   │   )                                                                 │
│   1473 │   │   │   │   │   │                                                                     │
│ ❱ 1474 │   │   │   │   │   │   self._model = self._load_state_dict(                              │
│   1475 │   │   │   │   │   │   │   model=model,                                                  │
│   1476 │   │   │   │   │   │   │   path=top_k_model_paths[0],                                    │
│   1477 │   │   │   │   │   │   │   prefix=prefix,                                                │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/autogluon/multimodal/predictor.py:2319 in                 │
│ _load_state_dict                                                                                 │
│                                                                                                  │
│   2316 │   │   strict: bool = True,                                                              │
│   2317 │   ):                                                                                    │
│   2318 │   │   if state_dict is None:                                                            │
│ ❱ 2319 │   │   │   state_dict = torch.load(path, map_locati

## 3. 성능평가

In [ ]:
# 모델 성능평가
# predictor.leaderboard(silent = True)

In [ ]:
# test_score = predictor.evaluate(test_data)
# print(test_score)

Predicting: 0it [00:00, ?it/s]

{'acc': 0.8466666666666667}


In [ ]:
# model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_data)

Predicting: 0it [00:00, ?it/s]

In [ ]:
submission['Category'] = model_pred

In [ ]:
submission = submission.astype({'Category':'int64'})

In [ ]:
submission

,Id,Category
0,710,1
1,487,0
2,136,1
3,44,1
4,627,0
...,...,...
244,702,1
245,500,0
246,818,0
247,584,0


In [ ]:
submission.to_csv(PATH+'submit/autoGluon_QA_new_unicode_2hours_submission_230306.csv', index=False)

In [ ]:
model = predictor.dump_model()

Model weights for hf_text are saved to /content/AutogluonModels/ag-20230306_125146/hf_text.
Tokenizer hf_text saved to /content/AutogluonModels/ag-20230306_125146/hf_text.


In [ ]:
import pickle

# 모델 객체 저장
with open(PATH+'model/autoGluon_QA_new_unicode_2hours_model_230306.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
from joblib import dump, load

# 모델 객체 저장
dump(model, PATH+'model/autoGluon_QA_new_unicode_2hours_model_230306.joblib')

# 저장된 모델 객체 로드
# model = load('model.joblib')

['/content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/model/autoGluon_QA_new_unicode_2hours_model_230306.joblib']